In [ ]:
from google.colab import files
uploaded = files.upload()

Saving df_upload.csv to df_upload.csv


KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
df = pd.read_csv("df_upload.csv")

In [ ]:
df.shape

(224988, 42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
df.shape
X = df.drop(['target'], axis = 1 )
y = df['target']


In [ ]:
X_10k = X.head(10000)
y_10k = y.head(10000)

In [ ]:
data = {"Model":[], "split_num":[],"iter_num":[], "AUC_score":[]}
wyniki = pd.DataFrame(data)

In [ ]:
#skalowanie danych
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_10k)
scaler.transform(X_10k)
X_10k = X_10k.drop(["Unnamed: 0"], axis = 1)


In [ ]:
X_10k.tail()

,state_LA,UPB_ratio_current,year_built,property_type_Seniors,property_value_acq,DTO_interest_rate,property_type_Multifamily,building_age,state_CA,property_unit_count_acq,...,DTB_interest_rate_origin,in_MBS,months_since_acq,original_term,60_days_DELIQ_ever,amortization_term,original_interest_rate,note_rate,payment_status_0,acq_date
9995,0.0,0.920191,1985.0,0.0,4.423077e+06,0.0,1.0,16.0,1.0,72.0,...,2.1,1.0,83.0,120.0,0.0,360.0,6.90,6.90,1.0,2001.419406
9996,0.0,0.905062,1985.0,0.0,4.423077e+06,0.0,1.0,16.0,1.0,72.0,...,2.1,1.0,95.0,120.0,0.0,360.0,6.90,6.90,1.0,2001.419406
9997,0.0,0.990870,1973.0,0.0,3.397219e+07,0.0,1.0,28.0,1.0,285.0,...,1.7,1.0,11.0,120.0,0.0,360.0,7.01,7.01,1.0,2001.419406
9998,0.0,0.981069,1973.0,0.0,3.397219e+07,0.0,1.0,28.0,1.0,285.0,...,1.7,1.0,23.0,120.0,0.0,360.0,7.01,7.01,1.0,2001.419406
9999,0.0,0.970740,1973.0,0.0,3.397219e+07,0.0,1.0,28.0,1.0,285.0,...,1.7,1.0,35.0,120.0,0.0,360.0,7.01,7.01,1.0,2001.419406


In [ ]:
X_10k.isna().sum()

state_LA                                   0
UPB_ratio_current                          0
year_built                                 0
property_type_Seniors                      0
property_value_acq                         0
DTO_interest_rate                          0
property_type_Multifamily                  0
building_age                               0
state_CA                                   0
property_unit_count_acq                    0
30_days_DELIQ_ever                         0
state_NC                                   0
occupancy                                  0
reporting_date                             0
loan_acq_LTV                               0
payment_status_1                           0
30_days_DELIQ_12M                          0
months_to_maturity_acq                     0
maturity_prolongation                      0
months_since_modification                  0
reference_rate                             0
id                                         0
months_sin

In [ ]:
# licze KNN dla 10 roznych parametrow
# 3 podzialy na 10 splitow
# iter_num to numer podzialu na splity
# split_num to numer splitu w danym podziale
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

knn_classifiers = [KNeighborsClassifier(n_neighbors = x) for x in range(3,22,2)]
kf_splitters = [StratifiedKFold(n_splits=10, random_state = x, shuffle=True) for x in range(1,4)]

wyniki_mod = []
wyniki_split_num = []
wyniki_iter_num = []
wyniki_auc_score = []

for i in range(0,10):
  model = knn_classifiers[i]
  wyniki_mod += ["knn" + str(i)] * 30
  for j in range(0,3):
    splitter = kf_splitters[j]
    scores = cross_val_score(model, X_10k, y_10k, cv = splitter, scoring = 'roc_auc')
    wyniki_split_num += [l for l in range(0,10)]
    wyniki_iter_num += [j] * 10
    wyniki_auc_score += list(scores)


In [ ]:
wyniki = pd.DataFrame({"Model":wyniki_mod, "split_num":wyniki_split_num,"iter_num":wyniki_iter_num, "AUC_score":wyniki_auc_score})
wyniki.head()

,Model,split_num,iter_num,AUC_score
0,AB0,0,0,0.935430
1,AB0,1,0,0.950043
2,AB0,2,0,0.894357
3,AB0,3,0,0.848051
4,AB0,4,0,0.912417


In [ ]:
# Random Forest dla 10 zestawów parametrów
# 10 podzialow na 10 splitow
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

params = [
          {'n_estimators': 50, 'max_depth': 4},
          {'n_estimators': 50, 'max_depth': 6},
          {'n_estimators': 50, 'max_depth': 8},
          {'n_estimators': 100, 'max_depth': 4},
          {'n_estimators': 100, 'max_depth': 6},
          {'n_estimators': 100, 'max_depth': 8},
          {'n_estimators': 200, 'max_depth': 4},
          {'n_estimators': 200, 'max_depth': 6},
          {'n_estimators': 200, 'max_depth': 8},
          {'n_estimators': 200, 'max_depth': 8, "criterion": 'entropy'},
]


RF_classifiers = [RandomForestClassifier(**params[i]) for i in range(0,10)]

wyniki_mod = []
wyniki_split_num = []
wyniki_iter_num = []
wyniki_auc_score = []

for i in range(0,10):
  model = RF_classifiers[i]
  wyniki_mod += ["RF" + str(i)] * 30
  for j in range(0,3):
    splitter = kf_splitters[j]
    scores = cross_val_score(model, X_10k, y_10k, cv = splitter, scoring = 'roc_auc')
    wyniki_split_num += [l for l in range(0,10)]
    wyniki_iter_num += [j] * 10
    wyniki_auc_score += list(scores)

wyniki_RF = pd.DataFrame({"Model":wyniki_mod, "split_num":wyniki_split_num,"iter_num":wyniki_iter_num, "AUC_score":wyniki_auc_score})
wyniki_RF.head(5)

,Model,split_num,iter_num,AUC_score
0,RF0,0,0,0.916452
1,RF0,1,0,0.897046
2,RF0,2,0,0.933988
3,RF0,3,0,0.857396
4,RF0,4,0,0.826029


In [ ]:
wyniki_RF = pd.DataFrame({"Model":wyniki_mod, "split_num":wyniki_split_num,"iter_num":wyniki_iter_num, "AUC_score":wyniki_auc_score})
wyniki_RF.head(5)

,Model,split_num,iter_num,AUC_score
0,RF0,0,0,0.916452
1,RF0,1,0,0.897046
2,RF0,2,0,0.933988
3,RF0,3,0,0.857396
4,RF0,4,0,0.826029


In [ ]:
# XGBoost dla 10 zestawów parametrów
# 3 podzialy na 10 splitow
from xgboost.sklearn import XGBClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

params = [
          {'subsample': 1, 'max_depth': 4, 'learning_rate': 0.1},
          {'subsample': 0.5, 'max_depth': 6, 'learning_rate': 0.3},
          {'subsample': 0.7, 'max_depth': 8, 'learning_rate': 0.5},
          {'subsample': 0.6, 'max_depth': 4, 'learning_rate': 0.1},
          {'subsample': 0.8, 'max_depth': 6, 'learning_rate': 0.3},
          {'subsample': 0.9, 'max_depth': 8, 'learning_rate': 0.5},
          {'subsample': 0.7, 'max_depth': 4, 'learning_rate': 0.1},
          {'subsample': 0.9, 'max_depth': 6, 'learning_rate': 0.3},
          {'subsample': 0.8, 'max_depth': 8, 'learning_rate': 0.5},
          {'subsample': 0.9, 'max_depth': 10, 'learning_rate': 0.3},
]


XGB_classifiers = [XGBClassifier(**params[i]) for i in range(0,10)]
XGB_splitters = [StratifiedKFold(n_splits=10, random_state = x, shuffle=True) for x in range(1,4)]

wyniki_mod = []
wyniki_split_num = []
wyniki_iter_num = []
wyniki_auc_score = []

for i in range(0,10):
  model = XGB_classifiers[i]
  wyniki_mod += ["XGB" + str(i)] * 30
  for j in range(0,3):
    splitter = kf_splitters[j]
    scores = cross_val_score(model, X_10k, y_10k, cv = splitter, scoring = 'roc_auc')
    wyniki_split_num += [i for i in range(0,10)]
    wyniki_iter_num += [j] * 10
    wyniki_auc_score += list(scores)

wyniki_XGB = pd.DataFrame({"Model":wyniki_mod, "split_num":wyniki_split_num,"iter_num":wyniki_iter_num, "AUC_score":wyniki_auc_score})
wyniki_XGB.head(5)

,Model,split_num,iter_num,AUC_score
0,XGB0,0,0,0.931416
1,XGB0,1,0,0.949731
2,XGB0,2,0,0.934611
3,XGB0,3,0,0.937192
4,XGB0,4,0,0.899957


In [ ]:
#Decision Tree
# dla 10 zestawów parametrów
# 3 podzialy na 10 splitow
from sklearn.tree import DecisionTreeClassifier

params = [
          {'criterion': "entropy", 'max_depth': 4, 'min_samples_split': 2},
          {'criterion': "entropy", 'max_depth': 6, 'min_samples_split': 5},
          {'criterion': "entropy", 'max_depth': 6, 'min_samples_split': 10},
          {'criterion': "entropy", 'max_depth': 4, 'min_samples_split': 20},
          {'criterion': "entropy", 'max_depth': 8, 'min_samples_split': 2},
          {'criterion': "entropy", 'max_depth': 8, 'min_samples_split': 10},
          {'criterion': "entropy", 'max_depth': 5, 'min_samples_split': 20},
          {'criterion': "entropy", 'max_depth': 6, 'min_samples_split': 2},
          {'criterion': "entropy", 'max_depth': 8, 'min_samples_split': 5},
          {'criterion': "entropy", 'max_depth': 7, 'min_samples_split': 10},
]


DT_classifiers = [DecisionTreeClassifier(**params[i]) for i in range(0,10)]

wyniki_mod = []
wyniki_split_num = []
wyniki_iter_num = []
wyniki_auc_score = []

for i in range(0,10):
  model = DT_classifiers[i]
  wyniki_mod += ["DT" + str(i)] * 30
  for j in range(0,3):
    splitter = kf_splitters[j]
    scores = cross_val_score(model, X_10k, y_10k, cv = splitter, scoring = 'roc_auc')
    wyniki_split_num += [i for i in range(0,10)]
    wyniki_iter_num += [j] * 10
    wyniki_auc_score += list(scores)

wyniki_DT = pd.DataFrame({"Model":wyniki_mod, "split_num":wyniki_split_num,"iter_num":wyniki_iter_num, "AUC_score":wyniki_auc_score})
wyniki_DT.head(5)

,Model,split_num,iter_num,AUC_score
0,DT0,0,0,0.798613
1,DT0,1,0,0.872652
2,DT0,2,0,0.826163
3,DT0,3,0,0.920313
4,DT0,4,0,0.800746


In [ ]:
for i in range(9):
  print(wyniki_DT[wyniki_DT.Model == ("DT" + str(i))]["AUC_score"].min())

0.7130904085772239
0.6430020283975659
0.6962836858881483
0.7130541871921182
0.6759272674587076
0.693126022913257
0.7164227760069545
0.5888510576644451
0.6894740654882643


In [ ]:
from sklearn.ensemble import AdaBoostClassifier 
params = [
          {'learning_rate': 0.25, 'n_estimators': 50 },
          {'learning_rate': 0.5, 'n_estimators': 50 },
          {'learning_rate': 1, 'n_estimators': 50 },
          {'learning_rate': 0.25, 'n_estimators': 100 },
          {'learning_rate': 0.5, 'n_estimators': 100 },
          {'learning_rate': 1, 'n_estimators': 100 },
          {'learning_rate': 0.25, 'n_estimators': 250 },
          {'learning_rate': 0.5, 'n_estimators': 250 },
          {'learning_rate': 1, 'n_estimators': 250 },
          {'learning_rate': 2, 'n_estimators': 250 }
]


AB_classifiers = [AdaBoostClassifier(**params[i]) for i in range(0,10)]

wyniki_mod = []
wyniki_split_num = []
wyniki_iter_num = []
wyniki_auc_score = []

for i in range(0,10):
  model = AB_classifiers[i]
  wyniki_mod += ["AB" + str(i)] * 30
  for j in range(0,3):
    splitter = kf_splitters[j]
    scores = cross_val_score(model, X_10k, y_10k, cv = splitter, scoring = 'roc_auc')
    wyniki_split_num += [i for i in range(0,10)]
    wyniki_iter_num += [j] * 10
    wyniki_auc_score += list(scores)

wyniki_AB = pd.DataFrame({"Model":wyniki_mod, "split_num":wyniki_split_num,"iter_num":wyniki_iter_num, "AUC_score":wyniki_auc_score})
wyniki_AB.head(5)

,Model,split_num,iter_num,AUC_score
0,AB0,0,0,0.935430
1,AB0,1,0,0.950043
2,AB0,2,0,0.894357
3,AB0,3,0,0.848051
4,AB0,4,0,0.912417


In [ ]:
for i in range(9):
  print(wyniki_AB[wyniki_AB.Model == ("AB" + str(i))]["AUC_score"].mean())

0.8951768673306371
0.8786592715231961
0.870424503962655
0.8900884892587884
0.875090605499306
0.8623006019007486
0.8789100184912838
0.8669768301436117
0.8659915340367199


In [ ]:
wyniki_all = pd.concat([wyniki, wyniki_RF, wyniki_XGB, wyniki_DT, wyniki_AB])
wyniki_all.head()

,Model,split_num,iter_num,AUC_score
0,AB0,0,0,0.935430
1,AB0,1,0,0.950043
2,AB0,2,0,0.894357
3,AB0,3,0,0.848051
4,AB0,4,0,0.912417


In [ ]:
wyniki_all.shape

(1500, 4)

In [ ]:
from google.colab import drive
drive.mount('drive')
wyniki_all.to_csv('wyniki_all.csv', index = False)
!cp wyniki_all.csv "drive/My Drive/"

Mounted at drive
